In [ ]:
import sqlite3

# Connect to the database
db_path = '/content/sample_data/Blooms.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

print("Table Summary (One row per table):\n")
print(f"{'Table Name':<12} | {'Row Count':<10} | Columns")
print("-" * 80)

# For each table, get column names and row count
for table in tables:
    # Get row count
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    row_count = cursor.fetchone()[0]

    # Get column names
    cursor.execute(f"PRAGMA table_info({table});")
    columns = [col[1] for col in cursor.fetchall()]
    column_str = ', '.join(columns)

    # Display in row
    print(f"{table:<12} | {row_count:<10} | {column_str}")

# Close connection
conn.close()


Table Summary (One row per table):

Table Name   | Row Count  | Columns
--------------------------------------------------------------------------------
FlanCSV      | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density
GPTCSV       | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density
LLamaCSV     | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density
NLPCSV       | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density
GTCSV        | 100        | resume_path, person_name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density


In [ ]:
!pip install python-docx


In [ ]:
# step1_upload_and_extract_jd.py
from docx import Document
import os

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    return "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])

# Ask user to upload
jd_file_path = input("📂 Enter full path to your JD (.docx) file: ").strip()

if not os.path.exists(jd_file_path):
    print("❌ File does not exist. Exiting.")
    exit()

jd_text = extract_text_from_docx(jd_file_path)
print("\n📄 Extracted JD Text:\n")
print(jd_text[:1000] + "\n..." if len(jd_text) > 1000 else jd_text)


📂 Enter full path to your JD (.docx) file: /content/sample_data/Business Development Executive.docx

📄 Extracted JD Text:

Business Development Executive
About the Position
We are seeking proactive and driven Business Development Executives to join our admissions team. This role is instrumental in promoting our specialized Postgraduate, Master’s, and Executive Certification Programs designed for working professionals across industries. You will work alongside the Business Development Manager to generate leads, engage with prospective learners, and drive admissions conversions.
Job Role and Responsibilities
Proactively identify and engage potential candidates via inbound/outbound calls, cold calling, and email campaigns
Qualify leads based on eligibility and interest, and collaborate with the Business Development Manager for timely follow-ups and conversions
Recommend suitable programs to prospective learners based on their professional background and career goals
Maintain detailed and 

In [ ]:
# step2_extract_jd_features_manual.py
import re

def basic_feature_extraction(jd_text):
    jd_text_lower = jd_text.lower()

    # Basic heuristics (adjust if needed)
    experience = int(re.search(r'(\d+)\+?\s+years? of experience', jd_text_lower).group(1)) if re.search(r'(\d+)\+?\s+years? of experience', jd_text_lower) else 1
    tenure = 1  # Assume 1 year if not found
    certification = jd_text_lower.count("certification")
    publication = jd_text_lower.count("publication") + jd_text_lower.count("research paper")
    skill_keywords = ["python", "sql", "excel", "communication", "leadership", "project management", "data analysis"]
    skill_density = sum(1 for skill in skill_keywords if skill in jd_text_lower)

    return {
        "experience": experience,
        "tenure": tenure,
        "certification": certification,
        "publication": publication,
        "skill_density": skill_density
    }

# === Run this after extracting jd_text in Step 1
jd_features = basic_feature_extraction(jd_text)

print("\n✅ Extracted JD Features:")
for k, v in jd_features.items():
    print(f"{k:<15}: {v}")



✅ Extracted JD Features:
experience     : 1
tenure         : 1
certification  : 1
publication    : 0
skill_density  : 2


In [ ]:
# step3_normalize_weights.py

def normalize_weights(jd_features):
    total = sum(jd_features.values())
    if total == 0:
        print("❌ All feature values are zero. Cannot normalize.")
        return {k: 0 for k in jd_features}
    return {k: v / total for k, v in jd_features.items()}

jd_weights = normalize_weights(jd_features)

print("\n📊 Normalized JD Weights:")
for k, v in jd_weights.items():
    print(f"{k:<15}: {v:.3f}")



📊 Normalized JD Weights:
experience     : 0.200
tenure         : 0.200
certification  : 0.200
publication    : 0.000
skill_density  : 0.400


In [ ]:
# step4_compute_fitment.py
import sqlite3
import pandas as pd
import os

DB_PATH = r"/content/sample_data/Blooms.db"  # ✅ Update path if needed
OUTPUT_DIR = "/content/sample_data/fitment_reports"
os.makedirs(OUTPUT_DIR, exist_ok=True)

model_tables = ["FlanCSV", "GPTCSV", "LLamaCSV", "NLPCSV", "GTCSV"]

def compute_score(row, weights):
    return round(
        row.get("skill_density", 0) * weights.get("skill_density", 0) +
        row.get("total_experience", 0) * weights.get("experience", 0) +
        row.get("current_tenure", 0) * weights.get("tenure", 0) +
        row.get("certification_count", 0) * weights.get("certification", 0) +
        row.get("publication_count", 0) * weights.get("publication", 0),
        2
    )

def classify_fit(score_pct):
    if score_pct < 31:
        return "Low Fit"
    elif score_pct < 71:
        return "Medium Fit"
    else:
        return "Fit"

def process_table(table, conn, weights):
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)

    required = {
        "resume_path", "person_name", "total_experience", "current_tenure",
        "certification_count", "publication_count", "skill_density"
    }

    if not required.issubset(df.columns):
        print(f"⚠️ Skipping {table} — missing required columns.")
        return

    # Compute raw and normalized score
    df["raw_score"] = df.apply(lambda row: compute_score(row, weights), axis=1)
    max_score = df["raw_score"].max() or 1
    df["fitment_score"] = (df["raw_score"] / max_score * 100).round(2)
    df["fitment_level"] = df["fitment_score"].apply(classify_fit)

    # Save final 4-column report
    out_df = df[["resume_path", "person_name", "fitment_level", "fitment_score"]]
    out_file = os.path.join(OUTPUT_DIR, f"{table}_fitment.csv")
    out_df.to_csv(out_file, index=False)
    print(f"✅ Saved: {out_file}")

# === Run for all tables ===
conn = sqlite3.connect(DB_PATH)
print("\n📊 Computing fitment for all models...\n")
for table in model_tables:
    process_table(table, conn, jd_weights)
conn.close()



📊 Computing fitment for all models...

✅ Saved: /content/sample_data/fitment_reports/FlanCSV_fitment.csv
✅ Saved: /content/sample_data/fitment_reports/GPTCSV_fitment.csv
✅ Saved: /content/sample_data/fitment_reports/LLamaCSV_fitment.csv
✅ Saved: /content/sample_data/fitment_reports/NLPCSV_fitment.csv
✅ Saved: /content/sample_data/fitment_reports/GTCSV_fitment.csv


In [ ]:
import pandas as pd
import os
from functools import reduce

FOLDER = "/content/sample_data/fitment_reports"
tables = {
    "Flan": "FlanCSV_fitment.csv",
    "GPT": "GPTCSV_fitment.csv",
    "LLama": "LLamaCSV_fitment.csv",
    "NLP": "NLPCSV_fitment.csv",
    "GT": "GTCSV_fitment.csv"
}

dfs = []

for model, file in tables.items():
    path = os.path.join(FOLDER, file)
    if os.path.exists(path):
        df = pd.read_csv(path)[["resume_path", "fitment_score"]].copy()
        df.rename(columns={"fitment_score": f"{model}Score"}, inplace=True)
        dfs.append(df)
        print(f"✅ Loaded: {file}")
    else:
        print(f"❌ Missing: {file}")

# Merge all on resume_path
if dfs:
    merged_df = reduce(lambda left, right: pd.merge(left, right, on="resume_path", how="outer"), dfs)

    # Bring in person_name only from GTCSV
    gt_path = os.path.join(FOLDER, "GTCSV_fitment.csv")
    gt_person = pd.read_csv(gt_path)[["resume_path", "person_name"]]
    merged_df = pd.merge(merged_df, gt_person, on="resume_path", how="left")

    # Reorder columns
    cols = ["resume_path", "person_name", "FlanScore", "GPTScore", "LLamaScore", "NLPScore", "GTScore"]
    merged_df = merged_df[[col for col in cols if col in merged_df.columns]]

    out_path = os.path.join(FOLDER, "Combined_Fitment_By_Resume.csv")
    merged_df.to_csv(out_path, index=False)
    print(f"\n✅ Combined table saved to:\n{out_path}")
    print("\n📊 Sample:")
    print(merged_df.head())
else:
    print("❌ No fitment files loaded. Check your folder paths.")


✅ Loaded: FlanCSV_fitment.csv
✅ Loaded: GPTCSV_fitment.csv
✅ Loaded: LLamaCSV_fitment.csv
✅ Loaded: NLPCSV_fitment.csv
✅ Loaded: GTCSV_fitment.csv

✅ Combined table saved to:
/content/sample_data/fitment_reports/Combined_Fitment_By_Resume.csv

📊 Sample:
        resume_path      person_name  FlanScore  GPTScore  LLamaScore  \
0    Resume (1).pdf  Swapnil Morande     100.00      13.0        0.00   
1   Resume (10).pdf   Kamlesh Sayare      87.88      51.0       56.47   
2  Resume (102).pdf   NAVEEN KUMAR N      12.12       8.0        9.41   
3  Resume (103).pdf      Parul Runij      33.33       6.0       17.65   
4  Resume (105).pdf   Tarumani Raghu      18.18      25.0       14.12   

   NLPScore  GTScore  
0    100.00   100.00  
1     21.93    38.87  
2      4.46     8.91  
3      8.18    32.39  
4      7.81    12.15  
